# Tables

    CREATE [ OR REPLACE ]
        [ { [ { LOCAL | GLOBAL } ] TEMP | TEMPORARY | VOLATILE | TRANSIENT } ]
      TABLE [ IF NOT EXISTS ] <table_name>    
      (
        -- Column definition 
        -- Additional column definitions           
        -- Out-of-line constraints        
      )
    
      [ CLUSTER BY ( <expr> [ , <expr> , ... ] ) ]
      [ ENABLE_SCHEMA_EVOLUTION = { TRUE | FALSE } ]
      [ DATA_RETENTION_TIME_IN_DAYS = <integer> ]
      [ MAX_DATA_EXTENSION_TIME_IN_DAYS = <integer> ]
      [ CHANGE_TRACKING = { TRUE | FALSE } ]
      [ DEFAULT_DDL_COLLATION = '<collation_specification>' ]
      [ COPY GRANTS ]
      [ COPY TAGS ]
      [ COMMENT = '<string_literal>' ]
      [ [ WITH ] ROW ACCESS POLICY <policy_name> ON ( <col_name> [ , <col_name> ... ] ) ]
      [ [ WITH ] AGGREGATION POLICY <policy_name> [ ENTITY KEY ( <col_name> [ , <col_name> ... ] ) ] ]
      [ [ WITH ] JOIN POLICY <policy_name> [ ALLOWED JOIN KEYS ( <col_name> [ , ... ] ) ] ]
      [ [ WITH ] STORAGE LIFECYCLE POLICY <policy_name> ON ( <col_name> [ , <col_name> ... ] ) ]
      [ [ WITH ] TAG ( <tag_name> = '<tag_value>' [ , <tag_name> = '<tag_value>' , ... ] ) ]
      [ WITH CONTACT ( <purpose> = <contact_name> [ , <purpose> = <contact_name> ... ] ) ]




## Permanent

- Default
- Time Travel (up to 90 days on Enterprise+) 
- 7‑day Fail‑safe for disaster recovery. 
- Recomended for production data


In [ ]:
use sf_cert_prep.public;

CREATE OR REPLACE TABLE t_perm (id INT, v STRING);
INSERT INTO t_perm VALUES (1,'A'),(2,'B');


## Transient

- Persist until dropped 
- No Fail‑safe
- Time Travel 0–1 day
- Recommended for intermediate/staging data


In [ ]:
use sf_cert_prep.public;

-- Transient (no Fail-safe; Time Travel 0-1 day)
CREATE OR REPLACE TRANSIENT TABLE t_tran (id INT, v STRING);
INSERT INTO t_tran VALUES (1,'A'),(2,'B');



## Temporary

- session‑scoped
- auto‑dropped at session end
- no Fail‑safe.
- Useful for scratch/ETL steps

***Important*** you can have a temporary table with the same name as permanent/transient table. The Temporary table will take precedent.



In [ ]:
use sf_cert_prep.public;
-- Temporary (session-scoped)
CREATE OR REPLACE TEMPORARY TABLE t_temp (id INT, v STRING);
INSERT INTO t_temp VALUES (99,'TEMP');


In [ ]:
%%sql -r dataframe_4

use sf_cert_prep.public;
-- Name shadowing demo
CREATE OR REPLACE TEMPORARY TABLE t_perm (id INT, v STRING);  -- shadows the permanent
SELECT * FROM t_perm;  -- returns the temp table
--DROP TABLE IF EXISTS t_perm; -- drops the temp; permanent still exists


## External

- read‑only metadata layer over files in S3/Azure/GCS
- exposes a VALUE column + file pseudo‑columns
- typically slower than native tables
- consider a materialized view for speed



## Hybrid

 - row‑store with indexes for low‑latency OLTP‑like access
 - PRIMARY KEY required
 - General Available on AWS/Azure only
 - Not allowed as transient/temporary



## Directory Tables 
- Pseudo-table.
- Listing of files on a stage
- Check NoSQL notebook for more details

## Iceberg tables

- data + metadata stored in your cloud storage (via an external volume).
- No Fail‑safe
- Snowflake‑managed variant supports full DML
- external‑catalog variant requires metadata refreshes

In [ ]:
-- Pre-req: an EXTERNAL VOLUME to your cloud storage
CREATE OR REPLACE ICEBERG TABLE ic_orders (
  order_id INT, amount DOUBLE, created_ts TIMESTAMP_NTZ
)
CATALOG = 'SNOWFLAKE'
EXTERNAL_VOLUME = 'MY_EXT_VOL'
BASE_LOCATION = 'iceberg/ic_orders';

INSERT INTO ic_orders VALUES (1, 12.34, CURRENT_TIMESTAMP());
SELECT * FROM ic_orders;

## Dynamic

- always‑fresh” tables maintained by Snowflake from a defining query + TARGET_LAG
- simplify pipelines vs streams/tasks
- Have important conditions / limitations:
    - source table must have change tracking enabled
    - 

In [ ]:
-- Base table with change tracking + non-zero Time Travel (required for incremental mode)
CREATE OR REPLACE TABLE raw_events (id NUMBER, ts TIMESTAMP, payload VARIANT);
ALTER TABLE raw_events SET CHANGE_TRACKING = TRUE, DATA_RETENTION_TIME_IN_DAYS = 1;

-- Append some data
INSERT INTO raw_events SELECT SEQ4(), CURRENT_TIMESTAMP(), OBJECT_CONSTRUCT('k','v') FROM TABLE(GENERATOR(ROWCOUNT=>10));

-- Dynamic table that aggregates counts by day, with 5-min target lag
CREATE OR REPLACE DYNAMIC TABLE dt_events_daily (
  day DATE,
  cnt NUMBER
)
TARGET_LAG = '5 minutes'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = INCREMENTAL
AS
SELECT DATE_TRUNC('DAY', ts) AS day, COUNT(*) AS cnt
FROM raw_events
GROUP BY 1;

-- Query it like a normal table
SELECT * FROM dt_events_daily ORDER BY day DESC;